In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import librosa.display
import IPython.display as display
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool1D, BatchNormalization,Conv1D
import tensorflow as tf
from keras import Model
# from keras.applications import ResNet50
from tensorflow.keras.layers import AveragePooling2D,Lambda,ReLU,MultiHeadAttention,Embedding, LeakyReLU
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Attention,GlobalAveragePooling1D,MultiHeadAttention
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.preprocessing import StandardScaler
from math import floor
from math import pi
import warnings
import math
warnings.filterwarnings("ignore")
import pickle



In [2]:
path = '/kaggle/input/birdclef-2021/'
os.listdir(path)

['train_soundscape_labels.csv',
 'sample_submission.csv',
 'train_soundscapes',
 'train_short_audio',
 'test.csv',
 'train_metadata.csv',
 'test_soundscapes']

In [3]:
def read_ogg_file(path, file):
    """ Read ogg audio file and return numpay array and samplerate"""
    
    data, samplerate = sf.read(path+file)
    return data, samplerate


def plot_audio_file(data, samplerate):
    """ Plot the audio data"""
    
    sr = samplerate
    fig = plt.figure(figsize=(10,10))
    x = range(len(data))
    y = data
    plt.plot(x, y)
    plt.plot(x, y, color='red')
    plt.legend(loc='upper center')
    plt.grid()
    
    
def plot_spectrogram(data, samplerate):
    """ Plot spectrogram with mel scaling """
    
    sr = samplerate
    spectrogram = librosa.feature.melspectrogram(data, sr=sr)
    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    librosa.display.specshow(log_spectrogram, sr=sr, x_axis='time', y_axis='mel')

In [4]:
os.listdir('../input/mod-melspec-full-v2')

['train_metadata_enriched.csv',
 'label_ids.pickle',
 '__results__.html',
 'audio_images',
 '__notebook__.ipynb',
 '__output__.json',
 'custom.css']

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# class MixupImageDataGenerator():
#     def __init__(self, generator, directory, batch_size, img_height, img_width, alpha=0.2, subset=None):
#         """Constructor for mixup image data generator.
#         Arguments:
#             generator {object} -- An instance of Keras ImageDataGenerator.
#             directory {str} -- Image directory.
#             batch_size {int} -- Batch size.
#             img_height {int} -- Image height in pixels.
#             img_width {int} -- Image width in pixels.
#         Keyword Arguments:
#             alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
#             subset {str} -- 'training' or 'validation' if validation_split is specified in
#             `generator` (ImageDataGenerator).(default: {None})
#         """

#         self.batch_index = 0
#         self.batch_size = batch_size
#         self.alpha = alpha

#         # First iterator yielding tuples of (x, y)
#         self.generator1 = generator.flow_from_directory(directory,
#                                                         target_size=(
#                                                             img_height, img_width),
#                                                         class_mode="categorical",
#                                                         batch_size=batch_size,
#                                                         shuffle=True,
#                                                         )

#         # Second iterator yielding tuples of (x, y)
#         self.generator2 = generator.flow_from_directory(directory,
#                                                         target_size=(
#                                                             img_height, img_width),
#                                                         class_mode="categorical",
#                                                         batch_size=batch_size,
#                                                         shuffle=True,
#                                                         )

#         # Number of images across all classes in image directory.
#         self.n = self.generator1.samples
#         self.class_indices=self.generator1.class_indices

#     def reset_index(self):
#         """Reset the generator indexes array.
#         """

#         self.generator1._set_index_array()
#         self.generator2._set_index_array()

#     def on_epoch_end(self):
#         self.reset_index()

#     def reset(self):
#         self.batch_index = 0

#     def __len__(self):
#         # round up
#         return (self.n + self.batch_size - 1) // self.batch_size

#     def get_steps_per_epoch(self):
#         """Get number of steps per epoch based on batch size and
#         number of images.
#         Returns:
#             int -- steps per epoch.
#         """

#         return self.n // self.batch_size

#     def __next__(self):
#         """Get next batch input/output pair.
#         Returns:
#             tuple -- batch of input/output pair, (inputs, outputs).
#         """

#         if self.batch_index == 0:
#             self.reset_index()

#         current_index = (self.batch_index * self.batch_size) % self.n
#         if self.n > current_index + self.batch_size:
#             self.batch_index += 1
#         else:
#             self.batch_index = 0

#         # random sample the lambda value from beta distribution.
#         l = np.random.beta(self.alpha, self.alpha, self.batch_size)

#         X_l = l.reshape(self.batch_size, 1, 1, 1)
#         y_l = l.reshape(self.batch_size, 1)

#         # Get a pair of inputs and outputs from two iterators.
#         X1, y1 = self.generator1.next()
#         X2, y2 = self.generator2.next()

#         # Perform the mixup.
#         X = X1 * X_l + X2 * (1 - X_l)
#         y = y1 * y_l + y2 * (1 - y_l)
#         return X, y

#     def __iter__(self):
#         while True:
#             yield next(self)

In [7]:
train_gen=ImageDataGenerator()
val_gen=ImageDataGenerator()

In [8]:
# train_generator = MixupImageDataGenerator(generator=train_gen,
#                                           directory='../input/melspec-on-full-data/audio_images/train/',
#                                           batch_size=100,
#                                           img_height=224,
#                                           img_width=224,
#                                           )


In [9]:
train_generator = train_gen.flow_from_directory(
    '../input/mod-melspec-full-v2/audio_images/train/',
    target_size=(224,224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=128,
    shuffle=True,
#     seed=2021,
    save_to_dir=None,
    subset=None,
#     interpolation="nearest",
)

Found 108696 images belonging to 397 classes.


In [10]:
class_dict=train_generator.class_indices
with open('class_dictionary,pickle','wb') as f:
    pickle.dump(class_dict,f)

In [11]:
val_generator = val_gen.flow_from_directory(
    '../input/mod-melspec-full-v2/audio_images/val/',
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=128,

 
#     interpolation="nearest",
)

Found 30667 images belonging to 397 classes.


In [12]:
import keras.backend as K
def model_loss(a,gamma=2):
    def focal_loss(y_true,y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        FL1=-a*K.pow((1-y_pred),gamma)*y_true*K.log(y_pred)
        FL2=-(1-a)*(1-y_true)*(y_pred**gamma)*K.log(1-y_pred)
        loss=K.mean(FL1+FL2)
        return loss
    return focal_loss
    
   

In [13]:
def scheduler(epoch,lr):
    if epoch < 4:
        return lr
    elif epoch < 7 and epoch >=4:
        return lr*0.8
    else:
        return 0.9*lr

In [14]:
def f1_score(y_true,y_pred):
    TP=K.sum(K.round(K.clip(y_true*y_pred,0,1)),axis=1)
    TPFN=K.sum(K.round(K.clip(y_true,0,1)),axis=1)
    TPFP=K.sum(K.round(K.clip(y_pred,0,1)),axis=1)
    PREC=TP/(TPFP+K.epsilon())
    REC=TP/(TPFN+K.epsilon())
    f1=K.mean(2*PREC*REC/(PREC+REC+K.epsilon()))
    return f1


In [15]:
from tensorflow.keras.applications import DenseNet121, EfficientNetB0,ResNet50,DenseNet169
from tensorflow.keras.layers import Reshape

In [16]:
def sed_model(name):
    if 'efficientnetb0' in name:
        top_model=EfficientNetB0(include_top=False,weights="imagenet",input_tensor=Input(shape=(224,224,3)),input_shape=None)
    if 'densenet121' in name:
        top_model=DenseNet121(include_top=False,weights="imagenet",input_tensor=Input(shape=(224,224,3)),input_shape=None)
    if 'densenet169' in name:
        top_model=DenseNet169(include_top=False,weights="imagenet",input_tensor=Input(shape=(224,224,3)),input_shape=None)
    if 'resnet' in name:
        top_model=ResNet50(include_top=False,weights="imagenet",input_tensor=Input(shape=(224,224,3)),input_shape=None)
    
#     headModel=Dense(1024)(top_model.layers[-1].output)
#     headModel=LeakyReLU(alpha=0.3)(headModel)
#     headModel=Dropout(0.5)(headModel)
# #     headModel=Dense(64)(headModel)
# #     headModel=LeakyReLU(alpha=0.3)(headModel)
    headModel=Flatten()(top_model.layers[-1].output)
    output=Dense(397, activation="sigmoid")(headModel)
    model = Model(inputs=top_model.input, outputs=output)
    return model



In [17]:
epochs = 10
# cycles=2
# lernrate = 1e-3
# max_rate=0.01

In [18]:
def plot_training(history,name):
    fig, axs = plt.subplots(1, 2, figsize=(16, 4))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    loss = history.history['loss']
    loss_val = history.history['val_loss']
    epochs = range(1, len(loss)+1)
    axs[0].plot(epochs, loss, 'bo', label='loss_train')
    axs[0].plot(epochs, loss_val, 'ro', label='loss_val')
    axs[0].set_title('Value of the loss function')
    axs[0].set_xlabel('epochs')
    axs[0].set_ylabel('value of the loss function')
    axs[0].legend()
    axs[0].grid()
    acc = history.history['accuracy']
    acc_val = history.history['val_accuracy']
    axs[1].plot(epochs, acc, 'bo', label='accuracy_train')
    axs[1].plot(epochs, acc_val, 'ro', label='accuracy_val')
    axs[1].set_title('Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Value of accuracy')
    axs[1].legend()
    axs[1].grid()

#     f1 = history.history['f1_score']
#     f1_val = history.history['val_f1_score']
#     axs[2].plot(epochs, f1, 'bo', label='mic_avg_f1_score_train')
#     axs[2].plot(epochs, f1_val, 'ro', label='mic_avg_f1_score_val')
#     axs[2].set_title('Micro Average F1 Score')
#     axs[2].set_xlabel('Epochs')
#     axs[2].set_ylabel('Value of F1 score')
#     axs[2].legend()
#     axs[2].grid()


    plt.show()
#     fig.savefig('train_model_{}.png'.format(name))

In [19]:
models=['efficientnetb0','densenet121','resnet']

In [20]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [21]:
from tensorflow.keras.losses import BinaryCrossentropy
BCELoss=BinaryCrossentropy(
    from_logits=False, label_smoothing=0.01, 
    name='binary_crossentropy'
)

In [22]:
from tensorflow.keras.optimizers import Adam,SGD

In [23]:

# for model_name in models[:1]:
#     print("************ Model{} *************".format(model_name))
#     model=sed_model(model_name)
#     opt =Adam(learning_rate=0.001)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=2, min_lr=0.0001)
#     model.compile(opt,loss=BCELoss,metrics=['accuracy'])
#     model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./{}_model.h5'.format(model_name),monitor='val_accuracy',verbose=1,mode='max',save_best_only=True)
#     history=model.fit_generator(train_generator,validation_data=val_generator,epochs = epochs, workers=4,callbacks=[model_checkpoint_callback,reduce_lr],use_multiprocessing=True)
#     with open('history_{}.pickle'.format(model_name), 'wb') as file:
#         pickle.dump(history.history, file)
# #     plot_training(history, model_name)
#     del model

    

In [24]:
model_name='efficientnetb0'
print("************ Model{} *************".format(model_name))
model=sed_model(model_name)
opt =Adam(learning_rate=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=2, min_lr=0.0001)
model.compile(opt,loss=BCELoss,metrics=['accuracy'])
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./{}_model.h5'.format(model_name),monitor='val_accuracy',verbose=1,mode='max',save_best_only=True)
history=model.fit_generator(train_generator,validation_data=val_generator,epochs = epochs, workers=4,callbacks=[model_checkpoint_callback,reduce_lr],use_multiprocessing=True)
with open('history_{}.pickle'.format(model_name), 'wb') as file:
    pickle.dump(history.history, file)
#     plot_training(history, model_name)
del model



************ Modelefficientnetb0 *************
16711680/16705208 [==============================] - 0s 0us/step
Epoch 1/10
850/850 [==============================] - 955s 1s/step - loss: 0.0594 - accuracy: 0.1271 - val_loss: 0.0418 - val_accuracy: 0.3427

Epoch 00001: val_accuracy improved from -inf to 0.34275, saving model to ./efficientnetb0_model.h5
Epoch 2/10
850/850 [==============================] - 968s 1s/step - loss: 0.0394 - accuracy: 0.5411 - val_loss: 0.0418 - val_accuracy: 0.4081

Epoch 00002: val_accuracy improved from 0.34275 to 0.40806, saving model to ./efficientnetb0_model.h5
Epoch 3/10
850/850 [==============================] - 974s 1s/step - loss: 0.0366 - accuracy: 0.8095 - val_loss: 0.0419 - val_accuracy: 0.4145

Epoch 00003: val_accuracy improved from 0.40806 to 0.41445, saving model to ./efficientnetb0_model.h5
Epoch 4/10
850/850 [==============================] - 979s 1s/step - loss: 0.0344 - accuracy: 0.9672 - val_loss: 0.0400 - val_accuracy: 0.4737

Epoch 000